# Filtering and Scaling Depth Data

Congratulations on making it to this part of the project. Before we continue, lets review the overarching goals of Depth Mapping and how we plan on getting there.

First, we want to write a function that will take raw depth and RGB values from the Kinect and filter out the object we are trying to map.  For example, if we want to map a water bottle that is between 2 to 4 feet away from the Kinect, we want to filter out all depth values that are not within 2 to 4 feet.


After we filter out data not in our bounds, we want to scale our data from its original size (640 x 480) into the size of our cube (6 x 6).

We already learned how to write Kinect still frames into .csv files, and in this notebook we will first read the data into an array.

Import the math and numpy modules first:

In [236]:
import math
import numpy as np

### Reading Depth Data

Next, initialize an empty python list and read from depth_data.csv.  Notice how, in the for loop, we create a list for each line using line.split(',').  Each line in f is originally a string of numbers, for example: '878,877,657'.  line.split(',') creates a python list in the form \['878', '877', '657'].  We then need to convert the string values into integers, which is where list comprehension becomes useful.  Given an example python list named 'list', \[int(i) for i in list] will convert all of the values in 'list' to integers if possible.  (Lookup documentation for [File Input/Output](https://docs.python.org/3/tutorial/inputoutput.html) and [list comprehension](https://www.pythonforbeginners.com/basics/list-comprehensions-in-python) for more help)

In the cell below, given an empty list depth_data, read the data that you wrote to a .csv file in the previous challenge into depth_data.

In [237]:
depth_data = []
depth_file = 'C:\\Users\\kevin\\Desktop\\ECE 196\\DepthMapping\\test\\test_Depth.csv'
depth_data = np.loadtxt(depth_file, delimiter=',')
print(depth_data)




[[2047. 2047. 2047. ... 2047. 2047. 2047.]
 [2047. 2047. 2047. ... 2047. 2047. 2047.]
 [2047. 2047. 2047. ... 2047. 2047. 2047.]
 ...
 [2047. 2047. 2047. ... 2047. 2047. 2047.]
 [2047. 2047. 2047. ... 2047. 2047. 2047.]
 [2047. 2047. 2047. ... 2047. 2047. 2047.]]


Check to make sure that the length of depth_data and the length of the first element of depth_data are 480 and 640, respectively.  These are the dimensions of the Kinect's camera.

Example: 

len(depth_data) == 480

len(depth_data[0]) == 640

In [238]:
depth_data.shape

(480, 640)

Create a variable 'raw_depth_data' to save the original values of depth_data before we filter the array.

In [239]:
raw_depth_data = np.copy(depth_data)

### Filtering Algorithm

Now that we have read our csv file into an array, we want to set bounds for the data we wish to see.  For example, if we want to visualize a water bottle that is between 3 and 5 feet from the Kinect, then we will only want to see data values between 3-5 feet to eliminate any background noise in the data.  Later on we will be visualizing our data and we will see the difference between the filtered and unfiltered data sets. 

At this point we will want depth_data to be a numpy array.  This will allow us to utilize an assortment of numpy functions that will make our algorithm more efficient.  If you are unfamiliar with numpy, refer to its documentation [here](https://docs.scipy.org/doc/numpy-1.16.1/reference/index.html) or check out our numpy tutorial.

We set our upper and lower depth bounds to values in feet that will capture the object we are trying to map.

In [240]:
ft_lo_depth = 2   # You can set these variables to different values
ft_u_depth = 2.5

Using the distance formula given earlier, convert the values from feet to 11-bit depth values and assign the result to variables lo_depth and u_depth.

In [306]:
def feet_to_raw(x):
    x = x*0.3048
    return ((5685/2)*math.atan((2500/309)*x) - 13488231/4000)
lo_depth = feet_to_raw(ft_lo_depth)
u_depth = feet_to_raw(ft_u_depth)
print(feet_to_raw(28))
print(f'lo_depth = {lo_depth}')
print(f'u_depth = {u_depth}')



1051.7669966158455
lo_depth = 524.3055707994426
u_depth = 635.8448362155896


In [375]:
def in_range(x):
    if 524<=x<=635:
        return 'True'
    else:
        return ''

In [380]:
vec = np.vectorize(in_range)
con = vec(raw_depth_data)
print(con[lo+1,:])

['' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' ''
 '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' ''
 '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' ''
 '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' ''
 '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' ''
 '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' ''
 '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' ''
 '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' ''
 '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' ''
 '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' ''
 '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' ''
 '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' ''
 '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' ''
 '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' ''

In order to make our algorithm as efficient as possible, we want to avoid for loops that run through every value of our data.  Flatten depth_data using np.flatten

In [242]:
depth_data = depth_data.flatten()

Now, because our array is one-dimensional, we want to be able to find the coordinates where our depth values are between lo_depth and u_depth.  By coordinates, we mean the x and y position where the depth value would be if it were a 2-dimensional array.  For example, if a value were at the position (2,3) in (rows, columns) of a 2D array with 5 rows and 4 columns, try to figure out what its position would be in a 1D array.  As in, if the 2D array were flattened, what single number would represent index (2,3) in our flattened array.  

In [243]:
def get_coordinates(x,rows,cols):
    r = int(x/cols)
    c = x%cols
    return(r,c)

Using this example, extrapolate to the shape of raw_depth_data, the original 2D array, and figure out how to map the indices of raw_depth_data in a one dimensional array called 'coordinates'.

Now there are a variety of ways to filter a one dimensional array, but in any case, our filtering array of depth values will need to satisfy the condition of being between our set 'lo_depth' and 'u_depth' bounds.

First, find the values of the indices where the depth_data fits given upper and lower bounds, and wrap the resulting 1D list of values in a numpy array.

In [244]:
#coordinates = [get_coorsinates(i) if lo_depth<=depth_data[i]<=u_depth else 0 for i in range(len(depth_data))]
coordinates = [i for i in range(len(depth_data)) if lo_depth<=depth_data[i]<=u_depth]
len(coordinates)

4508

Next, figure out how to filter 'depth_data' such that it is between 'lo_depth' and 'u_depth'

In [245]:
filtered_depth_data = [i for i in depth_data if lo_depth<=i<=u_depth]

Once we have filtered out depth values that are not within our set upper and lower bounds, compare the length of the raw_depth_data with depth_data to make sure that depth_data is in fact smaller.  When finding the length of raw_depth_data, consider that it is a 2D array with all rows being the same length and figure out how to calculate the total number of values in a 2D array.

In [246]:
# Compare the lengths of depth_data with raw_depth_data.
print(f'length of raw data = {len(depth_data)}')
print(f'length of filtered data = {len(filtered_depth_data)}')

print(filtered_depth_data[1000])
print(depth_data[coordinates[1000]])

length of raw data = 307200
length of filtered data = 4508
630.0
630.0


Continue if your length of depth_data is considerably smaller than the number of values in raw_depth_data.

### Scaling Algorithm

Now that we have created a 1D array that contains our filtered depth values, we want to scale our data in order to display it on a cube.

Let look at one method to scale any given object.  Given the image below, how would we map the water bottle?

<img src="example_image.jpg">

Picture a 3D cube surrounding the water bottle.  In our scaling algorithm, this cube will have the width and height of largest axis of our object (in this case the y-axis/height), and the depth will be as given by our bounds.  Our original bounds for depth set above may not in fact capture this water bottle, but imagine that they do.  What we want to do is take our cube surrounding the water bottle and scale the cube into smaller numbers.

Essentially we are fixing new dimensions for our cube that ultimately enables us to map the image on a physical cube.

Now that we have some intuition for what we wish to accomplish with our scaling algorithm, lets figure out how to scale our cube into smaller dimensions. Lets review what our filtered data looks like.

In [247]:
depth_data

array([2047., 2047., 2047., ..., 2047., 2047., 2047.])

Each value in our 1D array is a depth value representing the distance from the Kinect to whatever object you place in front of the Kinect.

First, lets find the x and y coordinates for each depth (z) value so that we can eventually transform our x, y, and z to the dimensions of the cube.

In [248]:
x_arr = [get_coordinates(i, 480, 640)[1] for i in range(len(depth_data)) if lo_depth<=depth_data[i]<=u_depth]  # All the x-values (from 0 to 639) where depth_data fits the depth bounds
y_arr =  [get_coordinates(i, 480, 640)[0] for i in range(len(depth_data)) if lo_depth<=depth_data[i]<=u_depth]  # All the y-values (from 0 to 479) where depth_data fits the depth bounds
z_arr = [i for i in depth_data if lo_depth<=i<=u_depth]  # All the z or depth values (11-bit) where depth_data fits the depth bounds
#print(x_arr)
print(len(depth_data))
get_coordinates(307199, 480,640)

307200


(479, 639)







































































































We will store the depth data under a new variable to be used in visualizing the data later.  This will be done by creating a list of tuples using [set()](https://docs.python.org/2/library/sets.html) and [zip()](https://docs.python.org/3/library/functions.html#zip) functions.

In [249]:
raw_tuple_depth = list(zip(x_arr,y_arr,z_arr))
len(raw_tuple_depth) 

4508

Find bounds for each dimension (x, y, z) and store them under new variables.

In [374]:
r = max(x_arr)   # Rightmost x-value
le = min(x_arr)  # Leftmost x-value
u = min(y_arr)   # Upper or smallest y-value
lo = max(y_arr)  # Lower or largest y-value
b = max(z_arr)   # Maximum z-value
f = min(z_arr)   # Minimum z-value
print(r,le,u,lo,b,f)

390 0 170 408 635.0 589.0


We can now use the bounds of our object to scale our data.  Here is what this would look like in our example image:

<img src="example_image_pt2.jpg">

raw_tuple_depth = ...
raw_tuple_depth

In [251]:
x_range = r-le  # potential legnth of cube
y_range = lo-u  # potential length of cube
print(x_range)
print(y_range)

390
238


Looking at the image above, it intuitively makes sense to map our object by forming a square around the largest range between x and y.  In this case the y-range is largest.

In [252]:
max_range = max(x_range, y_range)   # find the maximum range between x and y.  A square will be formed around the maximum range which our object will be placed in
x_mid = int(le + x_range/2)    # calculate the midpoint of the x-axis of the object
y_mid = int(u + y_range/2)    # calculate the midpoint of the y-axis of the object
print(x_range)
print(x_mid, y_mid)

390
195 289


In [253]:
y_mid

289

In [254]:
x_half = x_range / 2  # half of the x_range of the object
x_half

195.0

Create a conditional below that will change upper bounds (u) or left bounds (le) based on if the x_range is larger or if the y_range is larger, respectively.  You can see what this will accomplish in the image below.

In [255]:
# Make an if else statement
if x_range>y_range:
    u = y_mid-max_range/2
else:
    le = x_mid-max_range/2
print(le,u)
print(y_mid, max_range, max_range/2)

0 94.0
289 390 195.0


Here is what our x-y mapping should look like:

<img src="example_image_pt3.jpg">

As for the z_range of our cube, we will use the distance between 'u_depth' and 'lo_depth'.

In [359]:
z_range = u_depth - lo_depth

In [360]:
z_range

111.53926541614692

Create a variable 'cube_length' that will represent the length of our cube for this project.

In [361]:
cube_length = 6

Now we have all the components we need to scale 'depth_data' into our cube's values.  The resulting x, y, and z arrays should have values between 0 and the cube_length.

Try drawing a picture for this section to figure out how to scale our 3D cube from one set of dimensions into another.  Consider the picture above.

Variables 'x_new', 'y_new', and 'z_new' will represent our x, y, and z values in the dimensions of our cube. Ideally, we want to create these arrays using matrix manipulation allowed by numpy.  This means creating arrays and applying arithmetic operations on them to avoid using for loops.  This will make our algorithm much more efficient.

Hint: You will need to create additional variables below to scale the arrays and avoid for loops.  [This](https://www.pluralsight.com/guides/overview-basic-numpy-operations) article details how to apply arithmetic operations between numpy arrays.

In [381]:
x_range

390

In [426]:
widthOfBins = int(x_range/cube_length)
x_mtx = np.asarray(x_arr[le:le+max_range])
x_mtx = x_mtx.reshape(6,widthOfBins)
x_avg = np.average(x_mtx, axis=1)
x_avg

array([308.89230769, 308.18461538, 307.53846154, 309.        ,
       308.66153846, 310.73846154])

In [425]:
x_new = np.divide(x_avg, x_range/cube_length)
print(x_new.astype(int))

[4 4 4 4 4 4]


In [382]:
# Create arrays x_new, y_new, and z_new that represent the x, y, and z values of our cube
y_new = ...
z_new = ... 

We will now be able to create our array of values that will map our object by combining x_new, y_new, and z_new arrays.  The output will be a list of tuples.

In [0]:
filtered_depth = ...
filtered_depth

As a final check, make sure that the sets of your x, y, and z arrays are within the values 0 to the length of the cube.

# Visualizing Data

In this section we will be using sklearn.cluster and plotly to visualize our data.

In [0]:
# Make sure you have these following packages installed
#from sklearn.cluster import SpectralClustering, KMeans
import plotly.offline as py
from plotly import graph_objs as go

Lets look at our raw_tuple_depth variable we created earlier.  We will first visualize our raw data and compare that to our mapped data.

In [0]:
raw_tuple_depth

Put the name of your fi

In [0]:
#plots = []
x, y, z = raw_tuple_depth[:, 0], raw_tuple_depth[:, 1], raw_tuple_depth[:, 2]
plots = [go.Scatter3d(x=x, y=y, z=z, mode='markers', marker=dict(size=2), connectgaps=False)] #,color=c[k]

file_name = ...  #the name of your file
py.plot(plots,filename=file_name,auto_open=True)

Now lets look at our updated 'filtered_depth'.

In [0]:
x, y, z = filtered_depth[:, 0], filtered_depth[:, 1], filtered_depth[:, 2]
plots = [go.Scatter3d(x=x, y=y, z=z, mode='markers', marker=dict(size=2), connectgaps=False)]
file_name = ... #the name of your file
py.plot(plots,filename=file_name,auto_open=True)

** If you want more information as to how these plots were generated, refer to [plotly](https://plot.ly/python/user-guide/) documentation